In [12]:
import pandas as pd
import numpy as np
import os
import sys
import gzip
import shutil
import json
import re
import tmalign_mapping
import Bio.PDB as bpdb

# Zip Status

In [2]:
# Zip status
zipped_status_path = "/nfs/turbo/lsa-tewaria/zipped_status.csv"
if not os.path.exists(zipped_status_path):
    list_mmcifs = os.listdir("/nfs/turbo/lsa-tewaria/mmCIF/")
    zipped_status = pd.DataFrame({"status" : [True for i in range(len(list_mmcifs))]}, index = list_mmcifs)
    zipped_status.to_csv(zipped_status_path)
else:
    zipped_status = pd.read_csv(zipped_status_path, index_col = 0)

# Uniprot to pdb mappings

In [13]:
# Load dataframe PDBs to Uniprot
uniprot_pdb_path = "/nfs/turbo/lsa-tewaria/uniprot_df_small.csv"
uniprot_df = pd.read_csv(uniprot_pdb_path)

# Scop data

In [14]:
# Load Scop data
scop_df = tmalign_mapping.read_scop_file()

# Single Uniprot ID - Naive Version. All to All TM align

In [5]:
# # Move PDBs to Uniprot folders
uniprot_path, zipped_status = tmalign_mapping.uniprot_to_pdb("CHEY_ECOLI", uniprot_df, zipped_status)

# Compare proteins in Uniprot folder
tmalign_mapping.compare_proteins_dir(uniprot_path)

# Clean TMalign output
tmalign_mapping.clean_TMalign_output(uniprot_path)

# Save zipped status
zipped_status.to_csv(zipped_status_path)

/home/srayan/conformer-gen/tmalign_mapping.py:114: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  tmalign_output = pd.read_csv(os.path.join(directory, raw_file), sep = "\t", skipfooter = 1)


# All Uniprot IDs - Naive version

**NOTE: DON'T RUN THE BELOW CODE CHUNK. IT WILL TAKE A LONG TIME**

In [ ]:
# Get Unique Uniprot IDs
uniprot_ids = uniprot_df["uniprot_id"].unique()  
for uniprot_id in uniprot_ids:
    # Find Uniprot data
    uniprot_json = tmalign_mapping.find_uniprot(uniprot_df, uniprot_id)

    # Move PDBs to Uniprot folders
    uniprot_path, zipped_status = tmalign_mapping.uniprot_to_pdb(uniprot_id, uniprot_df, zipped_status)

    # Compare proteins in Uniprot folder
    tmalign_mapping.compare_proteins_dir(uniprot_path)

    # Clean TMalign output
    tmalign_mapping.clean_TMalign_output(uniprot_path)
    

# Single Uniprot ID - Smart versions

In [7]:
uniprot_json = tmalign_mapping.find_uniprot(uniprot_df, "CO5_HUMAN")
domain_info = tmalign_mapping.get_domain_info_for_pdbs(uniprot_json, scop_df)
uniprot_path, zipped_status = tmalign_mapping.uniprot_to_pdb("CO5_HUMAN", uniprot_df, zipped_status)
tmalign_mapping.compare_proteins_domain("/nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN", domain_info)

Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/1cfa.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/4p39.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/5b71.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/5hcc.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/5hcd.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/5hce.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/6rpt.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/4p39.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/5b71.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/5hcc.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria

# All Uniprot IDs - Smart version

**NOTE: DON'T RUN THE BELOW CODE CHUNK. IT WILL TAKE A LONG TIME**

In [ ]:
# Get Unique Uniprot IDs
uniprot_ids = uniprot_df["uniprot_id"].unique()  

for uniprot_id in uniprot_ids:
    # Find Uniprot data
    uniprot_json = tmalign_mapping.find_uniprot(uniprot_df, uniprot_id)

    # Get the uniprot domain info
    domain_info = tmalign_mapping.get_domain_info_for_pdbs(uniprot_json, scop_df)

    # Move PDBs to Uniprot folders
    uniprot_path, zipped_status = tmalign_mapping.uniprot_to_pdb(uniprot_id, uniprot_df, zipped_status)

    # Compare proteins in Uniprot folder
    tmalign_mapping.compare_proteins_domain(uniprot_path, domain_info)

# Checking the zip function 

In [5]:
def zip_cif_folder(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".cif"):
            filename_new = os.path.join(directory, filename)
            os.remove(filename_new)

In [9]:
def zip_mmcif_dir(directory):
    for filename in os.listdir(directory):
        new_dir = os.path.join(directory, filename)
        if os.path.isdir(new_dir):
            zip_cif_folder(new_dir)

In [7]:
dir_path = "/nfs/turbo/lsa-tewaria/mmCIF/hq"
zip_cif_folder(dir_path)

In [10]:
zip_mmcif_dir("/nfs/turbo/lsa-tewaria/mmCIF")

# Single uniprot ID 

In [15]:
uniprot_json = tmalign_mapping.find_uniprot(uniprot_df, "CO5_HUMAN")
domain_info = tmalign_mapping.get_domain_info_for_pdbs(uniprot_json, scop_df)
uniprot_path= tmalign_mapping.uniprot_to_pdb_efficient("CO5_HUMAN", uniprot_df, write = False)
tmalign_mapping.compare_proteins_domain_efficient("/nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN", domain_info)

Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/1cfa.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/4p39.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/5b71.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/5hcc.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/5hcd.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/5hce.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/6rpt.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/4p39.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/5b71.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/5hcc.cif. Chain number 0.
Warning! Cannot parse file: /nfs/turbo/lsa-tewaria

# Single Uniprot ID - with cutting business, Comparing proteins with the same domains